In [2]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
from skimage import morphology, filters, segmentation, measure, transform
import matplotlib.patches as mpatches
from skimage.color import label2rgb
from sklearn import cluster

In [4]:
def display_randomImages(imageList):
    limit = len(imageList)
    choices = []
    i = 0;
    while(i < 12):
        temp = np.random.randint(limit - 1)
        while(temp in choices):
            temp = np.random.randint(limit - 1)
        choices.append(temp)
        i+=1
    
    plt.figure(figsize=(10,10))
    plt.subplot(141); plt.imshow(imageList[choices[0]], cmap='gray'); plt.title([choices[0]])
    plt.subplot(142); plt.imshow(imageList[choices[1]], cmap='gray'); plt.title([choices[1]])
    plt.subplot(143); plt.imshow(imageList[choices[2]], cmap='gray'); plt.title([choices[2]])
    plt.subplot(144); plt.imshow(imageList[choices[3]], cmap='gray'); plt.title([choices[3]])
    plt.show()
    
    plt.figure(figsize=(10,10))
    plt.subplot(141); plt.imshow(imageList[choices[4]], cmap='gray'); plt.title([choices[4]])
    plt.subplot(142); plt.imshow(imageList[choices[5]], cmap='gray'); plt.title([choices[5]])
    plt.subplot(143); plt.imshow(imageList[choices[6]], cmap='gray'); plt.title([choices[6]])
    plt.subplot(144); plt.imshow(imageList[choices[7]], cmap='gray'); plt.title([choices[7]])
    plt.show()

    
    plt.figure(figsize=(10,10))
    plt.subplot(141); plt.imshow(imageList[choices[8]], cmap='gray'); plt.title([choices[8]])
    plt.subplot(142); plt.imshow(imageList[choices[9]], cmap='gray'); plt.title([choices[9]])
    plt.subplot(143); plt.imshow(imageList[choices[10]], cmap='gray'); plt.title([choices[10]])
    plt.subplot(144); plt.imshow(imageList[choices[11]], cmap='gray'); plt.title([choices[11]])
    plt.show()


    

In [7]:
image = imageio.imread('tp4_ex1.png')

'''
    Phase de Pretraitement du pipeline
'''
# conversion en tons de gris et en float
image_grayscale = image.mean(axis=2).astype(float)

# normalisation de l'intensite
image_grayscale_norm = (image_grayscale / 255)

# inversion des intensite
image_grayscale_norm_inv = (1 - image_grayscale_norm)

'''
    Phase de segmentation du pipeline, extraction et sauvegarde dans une liste d'image
'''
# Appliquer un filtrage par ouverture-fermeture morphologiques pour effacer l'arriere plan, 
# et par la meme occasion remplir des trous eventuels dans nos lettres
# Creation d'un element structurant 3 x 3
h = np.ones((3,3))

# appliquer une ouverture avec notre element structurant afin de retirer les bruit de l'arriere plan
image_opening = morphology.opening(image_grayscale_norm_inv, h)
# appliquer une fermeture afin de remplir d'eventuels trous
image_closing =  morphology.closing(image_opening, h)
# seuillage par methode otsu
threshold = filters.threshold_otsu(image_closing)
image_segmented = image_closing > threshold

# retirer les lettres touchant la bordure
image_lucid_borders = segmentation.clear_border(image_segmented)

# Etiqueter (label) les lettres
image_labeled = measure.label(image_lucid_borders)


# apres differents tests, ce seuil permet de garder un maximum de lettre en ignorant la ponctuation
area_limit = 150

# Creer la liste des images de lettres
image_list = []

# boucler sur les lettres detectees
for region in measure.regionprops(image_labeled):
    if region.area >= area_limit:
        tempo = transform.resize(region.image, (24,24))
        image_list.append(tempo)

        
'''
    Cette section est la pour demontrer les differentes etapes du pipeline utilisees. Decommenter pour voir 
    le cheminement du pipeline.
    
plt.figure(figsize=(20,20))
plt.subplot(121); plt.imshow(image); plt.title('image originale')
plt.subplot(122); plt.imshow(image_grayscale, cmap='gray'); plt.title('image tons gris')
plt.show()

plt.figure(figsize=(20,20))
plt.subplot(121); plt.imshow(image_grayscale_norm ,cmap='gray'); plt.title('image tons gris normalise')
plt.subplot(122); plt.imshow(image_grayscale_norm_inv,cmap='gray' ); plt.title('image tons gris normalise inverse')
plt.show()

plt.figure(figsize=(20,20))
plt.subplot(121); plt.imshow(image_opening,cmap='gray' ); plt.title('opening')
plt.subplot(122); plt.imshow(image_closing,cmap='gray'); plt.title('closing')
plt.show()

plt.figure(figsize=(20,20))
plt.subplot(121); plt.imshow(image_segmented ,cmap='gray'); plt.title('image segmented')
plt.subplot(122); plt.imshow(image_lucid_borders ,cmap='gray'); plt.title('image segmented clean borders')

plt.show()
'''


# Appel de la fonction d'affichage
display_randomImages(image_list)
